1. Problem Statement

2. Hypothesis Generation

3. Getting the system ready and loading the data

4. Understanding the data

In [1]:
# Supress warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)


import pandas as pd
import numpy as np
import skimpy 

ModuleNotFoundError: No module named 'skimpy'

In [ ]:
df = pd.read_csv('../data/sales.csv')
df.head()

In [ ]:
dfRowCountStart = len(df.index)
dfRowCountStart

In [ ]:
df.describe()

The dataset contains a mix of numerical and categorical variables representing various attributes of loan applicants and their loan applications. The dataset appears to be suitable for analyzing factors influencing the sales price of a item compared to the retail price the views that they had for different items and categories.


In [ ]:
# Assessing data quality, completeness, and relevance
print(df.info())

In [ ]:
- The dataset contains ... entries (rows) and ... columns.
- Each column represents a different variable or feature.
- The variables have different data types:
- ... columns are of type float64, representing numerical variables (e.g., CoapplicantIncome, LoanAmount).
- ... column is of type int64, representing a numerical variable (e.g., ApplicantIncome).
- ... columns are of type object, representing categorical variables (e.g., Gender, Married, Education).
- ... are missing values in several columns:
Gender, Married, Dependents, Self_Employed, LoanAmount, Loan_Amount_Term, and Credit_History have some missing values.
- The target variable (Loan_Status) is categorical and has two classes: Y (Yes) and N (No).
- Other categorical variables include Gender, Married, Education, Self_Employed, and Property_Area.
- Numerical variables include ApplicantIncome, CoapplicantIncome, LoanAmount, Loan_Amount_Term, and Credit_History.

In [ ]:
#Identifying potential data issues and limitations.
from skimpy import skim
skim(df)

5. Exploratory Data Analysis

In [ ]:
# For Visualization
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sn
import pandas as pd

In [ ]:
df = pd.read_csv('../data/sales.csv')
df.head()

In [ ]:
df.tail()

 	i. Perform Univariate Analysis

In [ ]:
(
    df['price_retail']
          .astype('')
)

In [ ]:
import pandas as pd
import plotly.express as px

# Assuming df is your DataFrame containing the 'price_retail' column
labels = (df['price_retail']
          .astype('')

# Rename columns for clarity
labels.columns = ['price_retail', 'Count']

# Create figure using Plotly
fig = px.bar(
    data_frame=labels, 
    x='price_retail', 
    y='Count', 
    title='Class Imbalance', 
    color='price_retail'
)

# Add titles & Display figure
fig.update_layout(xaxis_title='Sales price', yaxis_title='Sales made')
fig.show()

 	ii. Perform Bivariate Analysis

Numeric Features

In [ ]:
df.select_dtypes('').nunique()

In [ ]:
# this will change depending on the awnser above
# Select features to plot
plot_cols = ['sales', 'views', 'price_cost','price_cost']

# Plot numeric features against target
plt.Figure(figsize=(3,4))
for col in plot_cols:
    fig = px.box(data_frame=df[plot_cols], x=col, color=df['price_retail'], title=f'BoxPlot for {col} Feature against the Target')
    fig.update_layout(xaxis_title=f'{col} Feature')
    fig.show()

Categorical Feature

In [ ]:
df.select_dtypes('object').nunique()

In [ ]:
# this will change depending on the awnser above
# Select features to plot
plot_cols = ['category_id', 'id']

# Plot numeric features against target
plt.Figure(figsize=(3,4))
for col in plot_cols:
    fig = px.box(data_frame=df[plot_cols], x=col, color=df['price_retail'], title=f'BoxPlot for {col} Feature against the Target')
    fig.update_layout(xaxis_title=f'{col} Feature')
    fig.show()

6. Missing value and outlier treatment

Any variation of data that is not applicable or any strange characters we will replace the data with null values

In [ ]:
df.replace(['NaN', 'N/A', 'NA', 'n/a', 'n.a.', 'N#A', 'n#a', '?'], 'other', inplace=True)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

7. Evaluation Metrics for classification problem

8. Model building: part 1 (Apply Deep Learning classification algorithm without step 9)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import mglearn
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
import numpy as np




In [ ]:
# Assuming Sales_df is your DataFrame and 'Sales' is the column to be imputed
Sales_df = df[['sales']]

# Impute missing values using the mean strategy
imp_num = SimpleImputer(strategy='mean')
loan_df_imputed = pd.DataFrame(
    imp_num.fit_transform(Sales_df),  # Impute missing values
    columns=Sales_df.columns  # Keep the original column name
)

# Assign the imputed values back to the original DataFrame
df['sales'] = loan_df_imputed['sales']


# check missing values in %age
missing_values = (
    df.isnull().sum()/len(df)*100
).astype(int)

print(f'Column\t\t\t% missing')
print(f'{"-"}'*35)
missing_values

9. Feature engineering

10. Model building: part 2 (Apply Deep Learning classification algorithm with step 9)

11. Model deployment - Dash app on https://www.render.com